# Dataset Creation

In [635]:
import pandas as pd
import numpy as np
import requests
import io
import wbgapi as wb
from bs4 import BeautifulSoup
import cache_magic #pip install ipython_cache
import country_converter as coco

# Femicide Data

### Femicide data is downloaded from github and loaded as df_fem

In [636]:
url = 'https://raw.githubusercontent.com/Tom-Whittington/Femicide_analysis/main/processed_data/original_femicide_data.csv'

In [637]:
%cache download = requests.get(url).content

loading cached value for variable 'download'. Time since pickling  148 days, 1:47:15.801017


In [638]:
df_fem = pd.read_csv(io.StringIO(download.decode('utf-8')),
                dtype={'iso3_code' : 'category',
                           'Region' : 'category',
                           'Subregion' : 'category',
                           'country' : 'category',
                           'Indicator' : 'category',
                           'Gender' : 'category',
                           'Source' : 'category',
                           'Unit' : 'category',
                          'Year' : 'string'})


In [639]:
df_fem

,Region,Subregion,country,iso3_code,Indicator,Disaggregation,Gender,Source,Unit,Year,Value,Footnote
0,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2009,1115,NaN
1,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,"Rate per 100,000 population",2009,3.93,NaN
2,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2010,983,NaN
3,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,"Rate per 100,000 population",2010,3.37,NaN
4,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2011,1231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24387,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,"Rate per 100,000 population",2006,8.82,NaN
24388,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,Count,2010,711,NaN
24389,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,"Rate per 100,000 population",2010,5.6,NaN
24390,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,Count,2012,981,NaN


In [640]:
df_fem.dtypes

Region            category
Subregion         category
country           category
iso3_code         category
Indicator         category
Disaggregation      object
Gender            category
Source            category
Unit              category
Year                string
Value               object
Footnote            object
dtype: object

### Columns are renamed

In [641]:
df_fem.columns = [col.lower() for col in df_fem.columns]
df_fem = df_fem.rename(columns={'iso3_code' : 'iso3'})
df_fem['year'] = pd.to_datetime(df_fem['year'], format='%Y')
df_fem

,region,subregion,country,iso3,indicator,disaggregation,gender,source,unit,year,value,footnote
0,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2009-01-01,1115,NaN
1,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,"Rate per 100,000 population",2009-01-01,3.93,NaN
2,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2010-01-01,983,NaN
3,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,"Rate per 100,000 population",2010-01-01,3.37,NaN
4,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2011-01-01,1231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24387,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,"Rate per 100,000 population",2006-01-01,8.82,NaN
24388,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,Count,2010-01-01,711,NaN
24389,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,"Rate per 100,000 population",2010-01-01,5.6,NaN
24390,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,Count,2012-01-01,981,NaN


In [642]:
df_fem

,region,subregion,country,iso3,indicator,disaggregation,gender,source,unit,year,value,footnote
0,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2009-01-01,1115,NaN
1,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,"Rate per 100,000 population",2009-01-01,3.93,NaN
2,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2010-01-01,983,NaN
3,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,"Rate per 100,000 population",2010-01-01,3.37,NaN
4,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Total (all ages),NSO,Count,2011-01-01,1231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24387,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,"Rate per 100,000 population",2006-01-01,8.82,NaN
24388,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,Count,2010-01-01,711,NaN
24389,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,"Rate per 100,000 population",2010-01-01,5.6,NaN
24390,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Total (all ages),WHO,Count,2012-01-01,981,NaN


### Filters out totals and only keeps male and female data

In [643]:
df_fem = df_fem.loc[df_fem['gender'] == 'Female']
df_fem

,region,subregion,country,iso3,indicator,disaggregation,gender,source,unit,year,value,footnote
8,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,Count,2015-01-01,93,NaN
9,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,"Rate per 100,000 population",2015-01-01,0.56,NaN
14,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,Count,2016-01-01,101,NaN
15,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,"Rate per 100,000 population",2016-01-01,0.59,NaN
20,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,Count,2017-01-01,133,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24285,Americas,Latin America and the Caribbean,Venezuela (Bolivarian Republic of),VEN,Homicide: # of victims,-,Female,CTS,"Rate per 100,000 population",2016-01-01,5.36,NaN
24290,Americas,Latin America and the Caribbean,Venezuela (Bolivarian Republic of),VEN,Homicide: # of victims,-,Female,CTS,Count,2017-01-01,788,NaN
24291,Americas,Latin America and the Caribbean,Venezuela (Bolivarian Republic of),VEN,Homicide: # of victims,-,Female,CTS,"Rate per 100,000 population",2017-01-01,5.31,NaN
24372,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Female,MD,Count,1990-01-01,183,NaN


### Removes killed by intimate partner disaggregation

In [644]:
df_fem

,region,subregion,country,iso3,indicator,disaggregation,gender,source,unit,year,value,footnote
8,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,Count,2015-01-01,93,NaN
9,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,"Rate per 100,000 population",2015-01-01,0.56,NaN
14,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,Count,2016-01-01,101,NaN
15,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,"Rate per 100,000 population",2016-01-01,0.59,NaN
20,Asia,Southern Asia,Afghanistan,AFG,Homicide: # of victims,-,Female,NSO,Count,2017-01-01,133,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24285,Americas,Latin America and the Caribbean,Venezuela (Bolivarian Republic of),VEN,Homicide: # of victims,-,Female,CTS,"Rate per 100,000 population",2016-01-01,5.36,NaN
24290,Americas,Latin America and the Caribbean,Venezuela (Bolivarian Republic of),VEN,Homicide: # of victims,-,Female,CTS,Count,2017-01-01,788,NaN
24291,Americas,Latin America and the Caribbean,Venezuela (Bolivarian Republic of),VEN,Homicide: # of victims,-,Female,CTS,"Rate per 100,000 population",2017-01-01,5.31,NaN
24372,Africa,Sub-Saharan Africa,Zimbabwe,ZWE,Homicide: # of victims,-,Female,MD,Count,1990-01-01,183,NaN


### Filters out counts and leaves only count, sorts values and converts value column to numeric

In [645]:
df_fem = df_fem[df_fem['unit'] != 'Rate per  100,000 population']
df_fem = df_fem.sort_values(['year', 'gender', 'iso3'])
df_fem['value'] = pd.to_numeric(df_fem['value'], errors='coerce')
df_fem['iso3'] = df_fem['iso3'].replace({'KOS' : 'XKX'})
df_fem

,region,subregion,country,iso3,indicator,disaggregation,gender,source,unit,year,value,footnote
1192,Americas,Latin America and the Caribbean,Aruba,ABW,Homicide: # of victims,-,Female,MD,Count,1990-01-01,0.0,NaN
898,Asia,Western Asia,Armenia,ARM,Homicide: # of victims,-,Female,UNECE,Count,1990-01-01,22.0,NaN
674,Americas,Latin America and the Caribbean,Antigua and Barbuda,ATG,Homicide: # of victims,-,Female,MD,Count,1990-01-01,1.0,NaN
1294,Oceania,Australia and New Zealand,Australia,AUS,Homicide: # of victims,-,Female,MD,Count,1990-01-01,138.0,NaN
1560,Europe,Western Europe,Austria,AUT,Homicide: # of victims,-,Female,MD,Count,1990-01-01,39.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
22436,Asia,Central Asia,Tajikistan,TJK,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,2020-01-01,4.0,IPFM - Intimate partner or family member
22846,Africa,Northern Africa,Tunisia,TUN,Homicide: # of victims,-,Female,External,Count,2020-01-01,115.0,NaN
23730,Americas,Northern America,United States of America,USA,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,FBI,Count,2020-01-01,1420.0,IPFM - Intimate partner or family member
24138,Oceania,Melanesia,Vanuatu,VUT,Homicide: # of victims,-,Female,External,Count,2020-01-01,1.0,NaN


In [646]:
df_IPV = df_fem[df_fem['disaggregation'].str.contains('Killed')]
df_IPV

,region,subregion,country,iso3,indicator,disaggregation,gender,source,unit,year,value,footnote
120,Europe,Southern Europe,Albania,ALB,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,2005-01-01,7.0,IPFM - Intimate partner or family member
992,Asia,Western Asia,Armenia,ARM,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,2005-01-01,2.0,IPFM - Intimate partner or family member
1390,Oceania,Australia and New Zealand,Australia,AUS,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,2005-01-01,70.0,IPFM - Intimate partner or family member
1654,Europe,Western Europe,Austria,AUT,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,2005-01-01,39.0,IPFM - Intimate partner or family member
2058,Americas,Latin America and the Caribbean,Bahamas,BHS,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,2005-01-01,4.0,IPFM - Intimate partner or family member
...,...,...,...,...,...,...,...,...,...,...,...,...
21990,Europe,Northern Europe,Sweden,SWE,Homicide: # of victims - perpetrator IP,Killed by Intimate partner,Female,CTS,Count,2020-01-01,13.0,IP - Intimate partner
21996,Europe,Northern Europe,Sweden,SWE,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,External,Count,2020-01-01,13.0,IPFM - Intimate partner or family member
22436,Asia,Central Asia,Tajikistan,TJK,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,2020-01-01,4.0,IPFM - Intimate partner or family member
23730,Americas,Northern America,United States of America,USA,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,FBI,Count,2020-01-01,1420.0,IPFM - Intimate partner or family member


In [647]:
df_fem = df_fem[df_fem['disaggregation'] == '-']
df_fem = df_fem.set_index(['iso3','year'])

In [648]:
df_fem

,,region,subregion,country,indicator,disaggregation,gender,source,unit,value,footnote
iso3,year,,,,,,,,,,
ABW,1990-01-01,Americas,Latin America and the Caribbean,Aruba,Homicide: # of victims,-,Female,MD,Count,0.0,NaN
ARM,1990-01-01,Asia,Western Asia,Armenia,Homicide: # of victims,-,Female,UNECE,Count,22.0,NaN
ATG,1990-01-01,Americas,Latin America and the Caribbean,Antigua and Barbuda,Homicide: # of victims,-,Female,MD,Count,1.0,NaN
AUS,1990-01-01,Oceania,Australia and New Zealand,Australia,Homicide: # of victims,-,Female,MD,Count,138.0,NaN
AUT,1990-01-01,Europe,Western Europe,Austria,Homicide: # of victims,-,Female,MD,Count,39.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
SVN,2020-01-01,Europe,Southern Europe,Slovenia,Homicide: # of victims,-,Female,Covid-19,Count,6.0,NaN
SWE,2020-01-01,Europe,Northern Europe,Sweden,Homicide: # of victims,-,Female,External,Count,25.0,NaN
TJK,2020-01-01,Asia,Central Asia,Tajikistan,Homicide: # of victims,-,Female,CTS,Count,10.0,NaN


# Economic data

## Worldbank data import

### Economic data is imported from worldbank API
1. NY.GNP.PCAP.CD represents gross national income per capita

2. SI.POV.GINI represents the gini coefficient of each country

3. Years 2000-2023 are selected

4. Labels also returns country code

5. skipBlanks is turned off so we have a row for each year (easier merging)

6. each series is added to column

7. skipAggs returns only countries and not regions/aggregations

8. Result is cached as can be slow to return

In [649]:
%cache df_wb = wb.data.DataFrame(['NY.GNP.PCAP.CD','SI.POV.GINI', 'SP.POP.TOTL.FE.IN', 'SH.STA.SUIC.FE.P5', 'SH.MMR.DTHS', 'VC.IHR.PSRC.FE.P5'], time=range(1990,2023), labels=True, skipBlanks =False, columns='series', skipAggs=True)

loading cached value for variable 'df_wb'. Time since pickling  1 day, 7:15:48.231817


### Index is reset, columns renamed and converted to datetime format. Extra time column is dropped and index is set to country code and year

In [650]:
df_wb = df_wb.reset_index()

df_wb = df_wb.rename(columns={'economy':'iso3',
                        'Time':'year',
                        'NY.GNP.PCAP.CD':'gnipc',
                        'SI.POV.GINI':'gini',
                        'Country':'country',
                        'SP.POP.TOTL.FE.IN' : 'fem_pop',
                        'SH.STA.SUIC.FE.P5' : 'suicide_rate',
                        'SH.MMR.DTHS' : 'mat_deaths',
                        'VC.IHR.PSRC.FE.P5' : 'femicide_rate'})

df_wb['year'] = pd.to_datetime(df_wb['year'], format='%Y')

df_wb['gnipc'] = pd.to_numeric(df_wb['gnipc'], errors='coerce')
df_wb['gini'] = pd.to_numeric(df_wb['gini'], errors='coerce')

df_wb = df_wb.drop(columns='time')

df_wb = df_wb.set_index(['iso3', 'year'])
df_wb

country   gnipc  mat_deaths  suicide_rate  gini  \
iso3 year                                                              
ZWE  2022-01-01     Zimbabwe  1500.0         NaN           NaN   NaN   
     2021-01-01     Zimbabwe  1530.0         NaN           NaN   NaN   
     2020-01-01     Zimbabwe  1460.0      1700.0           NaN   NaN   
     2019-01-01     Zimbabwe  1450.0      1900.0           8.8  50.3   
     2018-01-01     Zimbabwe  1550.0      1700.0           8.7   NaN   
...                      ...     ...         ...           ...   ...   
AFG  1994-01-01  Afghanistan     NaN         NaN           NaN   NaN   
     1993-01-01  Afghanistan     NaN         NaN           NaN   NaN   
     1992-01-01  Afghanistan     NaN         NaN           NaN   NaN   
     1991-01-01  Afghanistan     NaN         NaN           NaN   NaN   
     1990-01-01  Afghanistan     NaN         NaN           NaN   NaN   

                   fem_pop  femicide_rate  
iso3 year                                  
ZWE  2022-01-01  8614935.0            NaN  
     2021-01-01  8449834.0       1.869859  
     2020-01-01  8284447.0       1.762339  
     2019-01-01  8122618.0       2.055987  
     2018-01-01  7966181.0       2.046150  
...                    ...            ...  
AFG  1994-01-01  7722096.0            NaN  
     1993-01-01  7000119.0            NaN  
     1992-01-01  6028939.0            NaN  
     1991-01-01  5372208.0            NaN  
     1990-01-01  5346409.0            NaN  

[7161 rows x 7 columns]

In [651]:
df_wb['fem_pop'] = df_wb['fem_pop'].astype('int64')

In [652]:
df_wb.isna().sum()

country             0
gnipc            1108
mat_deaths       3276
suicide_rate     3501
gini             5353
fem_pop             0
femicide_rate    4622
dtype: int64

In [653]:
df_wb

country   gnipc  mat_deaths  suicide_rate  gini  fem_pop  \
iso3 year                                                                       
ZWE  2022-01-01     Zimbabwe  1500.0         NaN           NaN   NaN  8614935   
     2021-01-01     Zimbabwe  1530.0         NaN           NaN   NaN  8449834   
     2020-01-01     Zimbabwe  1460.0      1700.0           NaN   NaN  8284447   
     2019-01-01     Zimbabwe  1450.0      1900.0           8.8  50.3  8122618   
     2018-01-01     Zimbabwe  1550.0      1700.0           8.7   NaN  7966181   
...                      ...     ...         ...           ...   ...      ...   
AFG  1994-01-01  Afghanistan     NaN         NaN           NaN   NaN  7722096   
     1993-01-01  Afghanistan     NaN         NaN           NaN   NaN  7000119   
     1992-01-01  Afghanistan     NaN         NaN           NaN   NaN  6028939   
     1991-01-01  Afghanistan     NaN         NaN           NaN   NaN  5372208   
     1990-01-01  Afghanistan     NaN         NaN           NaN   NaN  5346409   

                 femicide_rate  
iso3 year                       
ZWE  2022-01-01            NaN  
     2021-01-01       1.869859  
     2020-01-01       1.762339  
     2019-01-01       2.055987  
     2018-01-01       2.046150  
...                        ...  
AFG  1994-01-01            NaN  
     1993-01-01            NaN  
     1992-01-01            NaN  
     1991-01-01            NaN  
     1990-01-01            NaN  

[7161 rows x 7 columns]

In [654]:
df_wb['suicide_count'] =round((df_wb['suicide_rate'] / 100000) * df_wb['fem_pop'], 0)
df_wb['femicide_count'] = round((df_wb['femicide_rate'] / 100000) * df_wb['fem_pop'], 0)
df_wb['source'] = 'WB'

In [655]:
df_wb

country   gnipc  mat_deaths  suicide_rate  gini  fem_pop  \
iso3 year                                                                       
ZWE  2022-01-01     Zimbabwe  1500.0         NaN           NaN   NaN  8614935   
     2021-01-01     Zimbabwe  1530.0         NaN           NaN   NaN  8449834   
     2020-01-01     Zimbabwe  1460.0      1700.0           NaN   NaN  8284447   
     2019-01-01     Zimbabwe  1450.0      1900.0           8.8  50.3  8122618   
     2018-01-01     Zimbabwe  1550.0      1700.0           8.7   NaN  7966181   
...                      ...     ...         ...           ...   ...      ...   
AFG  1994-01-01  Afghanistan     NaN         NaN           NaN   NaN  7722096   
     1993-01-01  Afghanistan     NaN         NaN           NaN   NaN  7000119   
     1992-01-01  Afghanistan     NaN         NaN           NaN   NaN  6028939   
     1991-01-01  Afghanistan     NaN         NaN           NaN   NaN  5372208   
     1990-01-01  Afghanistan     NaN         NaN           NaN   NaN  5346409   

                 femicide_rate  suicide_count  femicide_count source  
iso3 year                                                             
ZWE  2022-01-01            NaN            NaN             NaN     WB  
     2021-01-01       1.869859            NaN           158.0     WB  
     2020-01-01       1.762339            NaN           146.0     WB  
     2019-01-01       2.055987          715.0           167.0     WB  
     2018-01-01       2.046150          693.0           163.0     WB  
...                        ...            ...             ...    ...  
AFG  1994-01-01            NaN            NaN             NaN     WB  
     1993-01-01            NaN            NaN             NaN     WB  
     1992-01-01            NaN            NaN             NaN     WB  
     1991-01-01            NaN            NaN             NaN     WB  
     1990-01-01            NaN            NaN             NaN     WB  

[7161 rows x 10 columns]

In [656]:
df_wb

country   gnipc  mat_deaths  suicide_rate  gini  fem_pop  \
iso3 year                                                                       
ZWE  2022-01-01     Zimbabwe  1500.0         NaN           NaN   NaN  8614935   
     2021-01-01     Zimbabwe  1530.0         NaN           NaN   NaN  8449834   
     2020-01-01     Zimbabwe  1460.0      1700.0           NaN   NaN  8284447   
     2019-01-01     Zimbabwe  1450.0      1900.0           8.8  50.3  8122618   
     2018-01-01     Zimbabwe  1550.0      1700.0           8.7   NaN  7966181   
...                      ...     ...         ...           ...   ...      ...   
AFG  1994-01-01  Afghanistan     NaN         NaN           NaN   NaN  7722096   
     1993-01-01  Afghanistan     NaN         NaN           NaN   NaN  7000119   
     1992-01-01  Afghanistan     NaN         NaN           NaN   NaN  6028939   
     1991-01-01  Afghanistan     NaN         NaN           NaN   NaN  5372208   
     1990-01-01  Afghanistan     NaN         NaN           NaN   NaN  5346409   

                 femicide_rate  suicide_count  femicide_count source  
iso3 year                                                             
ZWE  2022-01-01            NaN            NaN             NaN     WB  
     2021-01-01       1.869859            NaN           158.0     WB  
     2020-01-01       1.762339            NaN           146.0     WB  
     2019-01-01       2.055987          715.0           167.0     WB  
     2018-01-01       2.046150          693.0           163.0     WB  
...                        ...            ...             ...    ...  
AFG  1994-01-01            NaN            NaN             NaN     WB  
     1993-01-01            NaN            NaN             NaN     WB  
     1992-01-01            NaN            NaN             NaN     WB  
     1991-01-01            NaN            NaN             NaN     WB  
     1990-01-01            NaN            NaN             NaN     WB  

[7161 rows x 10 columns]

In [657]:
df_wb_femicide =df_wb.reset_index()
df_wb_femicide = df_wb_femicide[['iso3', 'year', 'femicide_count', 'source']]
df_wb_femicide = df_wb_femicide.set_index(['iso3', 'year'])
df_wb_femicide

femicide_count source
iso3 year                             
ZWE  2022-01-01             NaN     WB
     2021-01-01           158.0     WB
     2020-01-01           146.0     WB
     2019-01-01           167.0     WB
     2018-01-01           163.0     WB
...                         ...    ...
AFG  1994-01-01             NaN     WB
     1993-01-01             NaN     WB
     1992-01-01             NaN     WB
     1991-01-01             NaN     WB
     1990-01-01             NaN     WB

[7161 rows x 2 columns]

In [658]:
df_wb_femicide = df_wb_femicide.dropna()
df_wb_femicide

femicide_count source
iso3 year                             
ZWE  2021-01-01           158.0     WB
     2020-01-01           146.0     WB
     2019-01-01           167.0     WB
     2018-01-01           163.0     WB
     1990-01-01           183.0     WB
...                         ...    ...
ALB  1994-01-01            12.0     WB
     1993-01-01            16.0     WB
     1992-01-01            14.0     WB
AFG  2019-01-01           238.0     WB
     2018-01-01           261.0     WB

[2539 rows x 2 columns]

Filter out all rows that are already in the previous dataset. Important to note that some years have differences in the values and frequently 2021 is significantly different (maybe due to reporting being done mid year?)

In [659]:
df_wb_femicide = df_wb_femicide[~df_wb_femicide.index.isin(df_fem.index)]
df_wb_femicide = df_wb_femicide.rename(columns={'femicide_count' : 'value'})
df_wb_femicide

value source
iso3 year                    
ZWE  2021-01-01  158.0     WB
     2020-01-01  146.0     WB
     2019-01-01  167.0     WB
     2018-01-01  163.0     WB
PSE  2021-01-01    9.0     WB
...                ...    ...
DZA  2018-01-01   86.0     WB
     2017-01-01   56.0     WB
     2016-01-01   75.0     WB
ALB  2021-01-01   12.0     WB
AFG  2019-01-01  238.0     WB

[241 rows x 2 columns]

In [660]:
df_fem = pd.concat([df_fem, df_wb_femicide])
df_fem = df_fem.reset_index()
df_fem[['gender', 'disaggregation', 'indicator', 'unit']] = ['Female', '-', 'Homicide: # of victims', 'Count']
df_fem = pd.concat([df_fem, df_IPV])
df_fem

,iso3,year,region,subregion,country,indicator,disaggregation,gender,source,unit,value,footnote
0,ABW,1990-01-01,Americas,Latin America and the Caribbean,Aruba,Homicide: # of victims,-,Female,MD,Count,0.0,NaN
1,ARM,1990-01-01,Asia,Western Asia,Armenia,Homicide: # of victims,-,Female,UNECE,Count,22.0,NaN
2,ATG,1990-01-01,Americas,Latin America and the Caribbean,Antigua and Barbuda,Homicide: # of victims,-,Female,MD,Count,1.0,NaN
3,AUS,1990-01-01,Oceania,Australia and New Zealand,Australia,Homicide: # of victims,-,Female,MD,Count,138.0,NaN
4,AUT,1990-01-01,Europe,Western Europe,Austria,Homicide: # of victims,-,Female,MD,Count,39.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
21990,SWE,2020-01-01,Europe,Northern Europe,Sweden,Homicide: # of victims - perpetrator IP,Killed by Intimate partner,Female,CTS,Count,13.0,IP - Intimate partner
21996,SWE,2020-01-01,Europe,Northern Europe,Sweden,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,External,Count,13.0,IPFM - Intimate partner or family member
22436,TJK,2020-01-01,Asia,Central Asia,Tajikistan,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,4.0,IPFM - Intimate partner or family member
23730,USA,2020-01-01,Americas,Northern America,United States of America,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,FBI,Count,1420.0,IPFM - Intimate partner or family member


In [661]:
converter = coco.CountryConverter()
df_fem['region'] = converter.convert(names=df_fem['iso3'], src="ISO3", to="continent")
df_fem['subregion'] = converter.convert(names=df_fem['iso3'], src="ISO3", to="UNregion")
df_fem['country'] = converter.convert(names=df_fem['iso3'], src="ISO3", to="name_short")

In [662]:
df_fem = df_fem.dropna(subset='value')
df_fem

,iso3,year,region,subregion,country,indicator,disaggregation,gender,source,unit,value,footnote
0,ABW,1990-01-01,America,Caribbean,Aruba,Homicide: # of victims,-,Female,MD,Count,0.0,NaN
1,ARM,1990-01-01,Asia,Western Asia,Armenia,Homicide: # of victims,-,Female,UNECE,Count,22.0,NaN
2,ATG,1990-01-01,America,Caribbean,Antigua and Barbuda,Homicide: # of victims,-,Female,MD,Count,1.0,NaN
3,AUS,1990-01-01,Oceania,Australia and New Zealand,Australia,Homicide: # of victims,-,Female,MD,Count,138.0,NaN
4,AUT,1990-01-01,Europe,Western Europe,Austria,Homicide: # of victims,-,Female,MD,Count,39.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
21990,SWE,2020-01-01,Europe,Northern Europe,Sweden,Homicide: # of victims - perpetrator IP,Killed by Intimate partner,Female,CTS,Count,13.0,IP - Intimate partner
21996,SWE,2020-01-01,Europe,Northern Europe,Sweden,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,External,Count,13.0,IPFM - Intimate partner or family member
22436,TJK,2020-01-01,Asia,Central Asia,Tajikistan,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,CTS,Count,4.0,IPFM - Intimate partner or family member
23730,USA,2020-01-01,America,Northern America,United States,Homicide: # of victims - perpetrator IPFM,Killed by Intimate partner or family member (I...,Female,FBI,Count,1420.0,IPFM - Intimate partner or family member


## CIA data

### Gets webpage from url

In [663]:
url = 'https://www.cia.gov/the-world-factbook/field/gini-index-coefficient-distribution-of-family-income/country-comparison'

%cache r = requests.get(url)

soup = BeautifulSoup(r.text, 'lxml')

loading cached value for variable 'r'. Time since pickling  148 days, 1:47:01.230914


### Finds first table on page, extracts headers from first row and then loops over table and creates dataframe from values

In [664]:
table = soup.findAll('table')[0]

headers = [x.text for x in table.findChildren('th')[1:]]

row_list = headers
table_list = []
for row in table.findChildren('tr'):

    for cell in row.findChildren('td')[1:]:
        row_list.append(cell.text)
    table_list.append(row_list)
    row_list = []

df_cia = pd.DataFrame(table_list)

### Adds gini label to first row and then promotes first row to headers 

In [665]:
df_cia.iloc[0,1]='gini'

df_cia.columns= df_cia.iloc[0,:]

df_cia = df_cia[1:]

df_cia

,Country,gini,Date of Information
1,South Africa,63.0,2014 est.
2,Namibia,59.1,2015 est.
3,Zambia,57.1,2015 est.
4,Central African Republic,56.2,2008 est.
5,Eswatini,54.6,2016 est.
...,...,...,...
173,Slovakia,25.2,2016 est.
174,Belarus,24.4,2020 est.
175,Slovenia,24.2,2017 est.
176,Faroe Islands,22.7,2013 est.


### iso3 codes are generated for CIA data set by the worldbank api, Nas are dropped, year and iso3 set as index ready for filling in blanks 

In [666]:
df_cia['iso3'] = wb.economy.coder(df_cia['Country'])
df_cia = df_cia.dropna(subset='iso3')
df_cia['year'] =df_cia['Date of Information'].str[:4]
df_cia = df_cia[['iso3', 'year', 'Country', 'gini']]
df_cia['year'] = pd.to_datetime(df_cia['year'], format='%Y')
df_cia = df_cia.set_index(['iso3', 'year'])
df_cia

/tmp/ipykernel_13466/25053258.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cia['year'] =df_cia['Date of Information'].str[:4]


,,Country,gini
iso3,year,,
ZAF,2014-01-01,South Africa,63.0
NAM,2015-01-01,Namibia,59.1
ZMB,2015-01-01,Zambia,57.1
CAF,2008-01-01,Central African Republic,56.2
SWZ,2016-01-01,Eswatini,54.6
...,...,...,...
ARM,2020-01-01,Armenia,25.2
SVK,2016-01-01,Slovakia,25.2
BLR,2020-01-01,Belarus,24.4


In [667]:
df_wb.isna().sum()

country              0
gnipc             1108
mat_deaths        3276
suicide_rate      3501
gini              5353
fem_pop              0
femicide_rate     4622
suicide_count     3501
femicide_count    4622
source               0
dtype: int64

### Fills in any empty cells in wb dataset with values from cia dataset ~17 extra countries. gini column set back to float

In [668]:
df_eco = df_wb.copy()

df_eco['gini'] = df_eco['gini'].fillna(df_cia['gini'])

df_eco = df_eco.sort_index()

df_eco['gini'] =df_eco['gini'].astype('float64')

df_eco

country    gnipc  mat_deaths  suicide_rate  gini  fem_pop  \
iso3 year                                                                     
ABW  1990-01-01     Aruba  11570.0         NaN           NaN   NaN    33480   
     1991-01-01     Aruba  12560.0         NaN           NaN   NaN    34657   
     1992-01-01     Aruba  13610.0         NaN           NaN   NaN    35941   
     1993-01-01     Aruba  14640.0         NaN           NaN   NaN    37137   
     1994-01-01     Aruba  16060.0         NaN           NaN   NaN    38437   
...                   ...      ...         ...           ...   ...      ...   
ZWE  2018-01-01  Zimbabwe   1550.0      1700.0           8.7   NaN  7966181   
     2019-01-01  Zimbabwe   1450.0      1900.0           8.8  50.3  8122618   
     2020-01-01  Zimbabwe   1460.0      1700.0           NaN   NaN  8284447   
     2021-01-01  Zimbabwe   1530.0         NaN           NaN   NaN  8449834   
     2022-01-01  Zimbabwe   1500.0         NaN           NaN   NaN  8614935   

                 femicide_rate  suicide_count  femicide_count source  
iso3 year                                                             
ABW  1990-01-01       0.000000            NaN             0.0     WB  
     1991-01-01            NaN            NaN             NaN     WB  
     1992-01-01            NaN            NaN             NaN     WB  
     1993-01-01            NaN            NaN             NaN     WB  
     1994-01-01            NaN            NaN             NaN     WB  
...                        ...            ...             ...    ...  
ZWE  2018-01-01       2.046150          693.0           163.0     WB  
     2019-01-01       2.055987          715.0           167.0     WB  
     2020-01-01       1.762339            NaN           146.0     WB  
     2021-01-01       1.869859            NaN           158.0     WB  
     2022-01-01            NaN            NaN             NaN     WB  

[7161 rows x 10 columns]

# Merging

### The dichotamised economic dataframe is merged with the femicide dataframe on the shared iso3 index

In [669]:
df_fem = df_fem.set_index(['iso3', 'year'])


In [670]:
df_fem.index.dtypes

iso3            object
year    datetime64[ns]
dtype: object

In [671]:
df = df_fem.merge(df_eco, how='left', left_index=True, right_index=True).rename(columns={'country_x' : 'country'}).drop(columns=['country_y'])
df = df.sort_index()
df

region       subregion   country               indicator  \
iso3 year                                                                    
ABW  1990-01-01  America       Caribbean     Aruba  Homicide: # of victims   
     1995-01-01  America       Caribbean     Aruba  Homicide: # of victims   
     1999-01-01  America       Caribbean     Aruba  Homicide: # of victims   
     2001-01-01  America       Caribbean     Aruba  Homicide: # of victims   
     2002-01-01  America       Caribbean     Aruba  Homicide: # of victims   
...                  ...             ...       ...                     ...   
ZWE  1990-01-01   Africa  Eastern Africa  Zimbabwe  Homicide: # of victims   
     2018-01-01   Africa  Eastern Africa  Zimbabwe  Homicide: # of victims   
     2019-01-01   Africa  Eastern Africa  Zimbabwe  Homicide: # of victims   
     2020-01-01   Africa  Eastern Africa  Zimbabwe  Homicide: # of victims   
     2021-01-01   Africa  Eastern Africa  Zimbabwe  Homicide: # of victims   

                disaggregation  gender source_x   unit  value footnote  \
iso3 year                                                                
ABW  1990-01-01              -  Female       MD  Count    0.0      NaN   
     1995-01-01              -  Female       MD  Count    0.0      NaN   
     1999-01-01              -  Female       MD  Count    1.0      NaN   
     2001-01-01              -  Female       MD  Count    2.0      NaN   
     2002-01-01              -  Female       MD  Count    1.0      NaN   
...                        ...     ...      ...    ...    ...      ...   
ZWE  1990-01-01              -  Female       MD  Count  183.0      NaN   
     2018-01-01              -  Female       WB  Count  163.0      NaN   
     2019-01-01              -  Female       WB  Count  167.0      NaN   
     2020-01-01              -  Female       WB  Count  146.0      NaN   
     2021-01-01              -  Female       WB  Count  158.0      NaN   

                   gnipc  mat_deaths  suicide_rate  gini    fem_pop  \
iso3 year                                                             
ABW  1990-01-01  11570.0         NaN           NaN   NaN    33480.0   
     1995-01-01  16930.0         NaN           NaN   NaN    39724.0   
     1999-01-01  18320.0         NaN           NaN   NaN    45050.0   
     2001-01-01  20510.0         NaN           NaN   NaN    47178.0   
     2002-01-01  19290.0         NaN           NaN   NaN    47831.0   
...                  ...         ...           ...   ...        ...   
ZWE  1990-01-01    740.0         NaN           NaN   NaN  5247850.0   
     2018-01-01   1550.0      1700.0           8.7   NaN  7966181.0   
     2019-01-01   1450.0      1900.0           8.8  50.3  8122618.0   
     2020-01-01   1460.0      1700.0           NaN   NaN  8284447.0   
     2021-01-01   1530.0         NaN           NaN   NaN  8449834.0   

                 femicide_rate  suicide_count  femicide_count source_y  
iso3 year                                                               
ABW  1990-01-01       0.000000            NaN             0.0       WB  
     1995-01-01       0.000000            NaN             0.0       WB  
     1999-01-01       2.219731            NaN             1.0       WB  
     2001-01-01       4.239264            NaN             2.0       WB  
     2002-01-01       2.090672            NaN             1.0       WB  
...                        ...            ...             ...      ...  
ZWE  1990-01-01       3.487142            NaN           183.0       WB  
     2018-01-01       2.046150          693.0           163.0       WB  
     2019-01-01       2.055987          715.0           167.0       WB  
     2020-01-01       1.762339            NaN           146.0       WB  
     2021-01-01       1.869859            NaN           158.0       WB  

[3797 rows x 19 columns]